##Loading the Data from GitHub

In [ ]:
#@title Loading the Data from GitHub
!git clone https://github.com/Pavun-KumarCH/License-Plate-Detection-yolov8-9-ORC.git

## Install Requirement

In [ ]:
#@title Install requirement
!pip install -q ultralytics
!pip install -q roboflow

## Loading The Data

In [ ]:
#@title Loading The Data

import pandas as pd
import os
from PIL import Image

# Load CSV
csv_file = '/content/License-Plate-Detection-yolov8-9-ORC/DATA SCIENTIST_ASSIGNMENT/Licplatesdetection_train.csv'
df = pd.read_csv(csv_file)

# Path to images
images_path = '/content/License-Plate-Detection-yolov8-9-ORC/DATA SCIENTIST_ASSIGNMENT/license_plates_detection_train'

# Create YOLO format labels directory
labels_path = 'labels'
os.makedirs(labels_path, exist_ok=True)

# Process each image
for _, row in df.iterrows():
    img_id = row['img_id']
    ymin, xmin, ymax, xmax = row['ymin'], row['xmin'], row['ymax'], row['xmax']
    img_path = os.path.join(images_path, img_id)

    # Get image dimensions
    with Image.open(img_path) as img:
        width, height = img.size

    # Convert to YOLO format
    x_center = (xmin + xmax) / 2 / width
    y_center = (ymin + ymax) / 2 / height
    bbox_width = (xmax - xmin) / width
    bbox_height = (ymax - ymin) / height

    # Write to YOLO format file
    label_file = os.path.join(labels_path, os.path.splitext(img_id)[0] + '.txt')
    with open(label_file, 'w') as f:
        f.write(f"0 {x_center} {y_center} {bbox_width} {bbox_height}\n")

## Spliting the Detection Data into train and validation

In [ ]:
#@title Spliting the Data into train and validation
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

# Paths
csv_file = '/content/License-Plate-Detection-yolov8-9-ORC/DATA SCIENTIST_ASSIGNMENT/Licplatesdetection_train.csv'
images_path = '/content/License-Plate-Detection-yolov8-9-ORC/DATA SCIENTIST_ASSIGNMENT/license_plates_detection_train'
labels_path = '/content/labels'

# Output directories for split data
train_images_path = '/content/Data/Detection/train/images'
val_images_path = '/content/Data/Detection/val/images'
train_labels_path = '/content/Data/Detection/train/labels'
val_labels_path = '/content/Data/Detection/val/labels'

# train_images_path = '/content/License-Plate-Detection-yolov8-9-ORC/Data/detection/train/images'
# val_images_path = '/content/License-Plate-Detection-yolov8-9-ORC/Data/detection/val/images'
# train_labels_path = '/content/License-Plate-Detection-yolov8-9-ORC/Data/detection/train/labels'
# val_labels_path = '/content/License-Plate-Detection-yolov8-9-ORC/Data/detection/val/labels'

os.makedirs(train_images_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)
os.makedirs(val_labels_path, exist_ok=True)

# Load CSV file
df = pd.read_csv(csv_file)

# Split image filenames into training and validation sets
img_ids = df['img_id'].unique()
train_img_ids, val_img_ids = train_test_split(img_ids, test_size=0.2, random_state=42)

# Function to copy files
def copy_files(img_ids, src_img_dir, src_lbl_dir, dest_img_dir, dest_lbl_dir):
    for img_id in img_ids:
        # Copy images
        img_src = os.path.join(src_img_dir, img_id)
        img_dest = os.path.join(dest_img_dir, img_id)
        shutil.copy(img_src, img_dest)

        # Copy labels
        lbl_src = os.path.join(src_lbl_dir, os.path.splitext(img_id)[0] + '.txt')
        lbl_dest = os.path.join(dest_lbl_dir, os.path.splitext(img_id)[0] + '.txt')
        if os.path.exists(lbl_src):
            shutil.copy(lbl_src, lbl_dest)
        else:
            print(f"Warning: Label file for {img_id} not found.")

# Copy training and validation files
copy_files(train_img_ids, images_path, labels_path, train_images_path, train_labels_path)
copy_files(val_img_ids, images_path, labels_path, val_images_path, val_labels_path)

print("Data splitting complete.")

# Note : Before Training the Model Make Sure to create dataset.yaml file and add the respective train and val images path



train: /content/Data/detection/train/images
---
val: /content/Data/detection/val/images
---
nc: 1
---
names: ['license_plate']


In [ ]:
#@title Importing the Model & Training
from ultralytics import YOLO

# Load the model
model = YOLO('yolov8n.pt')

# Train the model
model.train(data='/content/dataset.yaml', epochs=50, imgsz=640, batch=16)

 ## Model Evaluation on the Test Data Images & Saving the results

In [ ]:
#@title Model Evaluation on the Test Data Images saving the results

# Define paths
best_model_path = '/content/runs/detect/train2/weights/best.pt'  # Update with the correct path
test_images_path = '/content/License-Plate-Detection-yolov8-9-ORC/DATA SCIENTIST_ASSIGNMENT/test/test'
results_dir = '/content/test/results'

# Create the results directory if it doesn't exist
os.makedirs(results_dir, exist_ok=True)

# Load the best model
model = YOLO(best_model_path)

# Run predictions on the test images
results = model.predict(test_images_path)

# Save the results
for i, result in enumerate(results):
    # Create a filename for each result
    filename = os.path.join(results_dir, f'result_{i}.jpg')
    result.save(filename)  # Save the predictions to the results directory

print(f"Results saved to {results_dir}")


## Visualizing the Detection of License plate on Test Data

In [ ]:
#@title Visualizing the Detection of License plate on Test Data
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Define the results directory
results_dir = '/content/test/results'

# Get the list of files in the results directory
image_files = [f for f in os.listdir(results_dir) if f.endswith(('.jpg', '.png'))]

# Display the images
for image_file in image_files:
    img_path = os.path.join(results_dir, image_file)
    img = mpimg.imread(img_path)

    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.title(image_file)
    plt.axis('off')  # Hide axis
    plt.show()
